# Compute to Data for Generative Art
The 3-C2D-flow-ocean notebook provides a good first step towards getting your head around C2D, but it is not configured to the C2D flow data scientists might use when training an ML model. In this notebook and those that follow, we'll work on the same steps but this time specifically for the Cryptopunks generative art project. The steps are separated for modularity and to provide a template that can be used in subsequent projects

### Notes from the original C2D Notebook:
- **3. Alice publishes a dataset** is not needed here (this is relevant for teams who want to publish datasets, not teams who want to use existing datasets to train ML models)
- **4. Alice publishes an algorithm** is relevant (we're going to publish the ML model as an algorithm)
- **5. Bob acquires datatokens for data and algorithm** is only needed to get the datatokens for the data (we already own the tokens for the algorithm)
- **6. Bob starts a compute job** is relevant (we need to specify where we want to train the algorithm on. Initially, the simplest option is connecting to an AWS compute instance, later on to decentralized computer providers such as the Raven Protocol)
- **7. Bob monitors logs / algorithm output** is relevant

#### Additional steps not covered in C2D Notebook:
- testing the trained model
- running inference
- retraining the model (after change of architecture/hyperparameter tuning)
- publishing the trained model on the marketplace
- using the model from the marketplace to get a new Cryptopunk

In [2]:
from IPython.display import Image 
import _init_paths

You can publish an algorithm through the GUI. Check out our first hacking session where we run through [this](https://www.youtube.com/watch?v=AThhcQrjRQk&list=PLgIrgqrkZC93qCxZFx_kWzk2vFdvgJjJI&t=35m20s) process. Here we'll use the Ocean python library. 

We need to connect to the Ethereum network via an Ethereum node. We have set the config parameters for you in a config file. We are currently using [Infura](https://infura.io) for this but will be migrating to a full Ethereum Erigon node asap for increased decentralization. 

In [3]:
from ocean_lib.ocean.ocean import Ocean
from ocean_lib.config import Config

config = Config('config.ini')
ocean = Ocean(config)

print(f"config.network_url = '{config.network_url}'")
print(f"config.block_confirmations = {config.block_confirmations.value}")
print(f"config.metadata_cache_uri = '{config.metadata_cache_uri}'")
print(f"config.provider_url = '{config.provider_url}'")

config.network_url = 'https://rinkeby.infura.io/v3/d163c48816434b0bbb3ac3925d6c6c80'
config.block_confirmations = 0
config.metadata_cache_uri = 'https://aquarius.oceanprotocol.com'
config.provider_url = 'https://provider.rinkeby.oceanprotocol.com'


Next, export your private key from your metamask wallet. We highly recommend doing this with a wallet that has no real tokens in it (only Rinkeby tokens). For more info on private keys, see [this](https://github.com/oceanprotocol/ocean.py/blob/main/READMEs/wallets.md) from the ocean.py documentation: 

*The whole point of crypto wallets is to store private keys. Wallets have various tradeoffs of cost, convienence, and security. For example, hardware wallets tend to be more secure but less convenient and not free. It can also be useful to store private keys locally on your machine, for testing, though only with a small amount of value at stake (keep the risk down). Do not store your private keys on anything public, unless you want your tokens to disappear. For example, don't store your private keys in GitHub or expose them on frontend webpage code.*

With this in mind, you can directly load your private key into the notebook. We use an envvar rather than storing it in code that might be pushed to a repo. We copy this in for a new session (you may need to restart the notebook server). Here's how we export an environmental variable using an example key (replace this with your actual private key.). From your console:

```console
export MY_TEST_KEY=0xaefd8bc8725c4b3d15fbe058d0f58f4d852e8caea2bf68e0f73acb1aeec19baa
```

Now initialize your wallet:

In [6]:
import os
from ocean_lib.web3_internal.wallet import Wallet

wallet = Wallet(ocean.web3, os.getenv('MY_TEST_KEY'), transaction_timeout=20, block_confirmations=config.block_confirmations)

print(f"public address = '{wallet.address}'")

public address = '0x2338e4e94AEe1817701F65f2c751f7c844b0e43b'


In [7]:
from ocean_lib.web3_internal.currency import to_wei

# Publish ALG datatoken
ALG_datatoken = ocean.create_data_token('ARTGEN0', 'ARTGEN0', wallet, blob=ocean.config.metadata_cache_uri)
ALG_datatoken.mint(wallet.address, to_wei(100), wallet)
print(f"ALG_datatoken.address = '{ALG_datatoken.address}'")

ALG_datatoken.address = '0xE2e123115d5758Dd4C6F434E1c142e72ed8B2820'


In [9]:
from ocean_lib.web3_internal.currency import pretty_ether_and_wei
print(f"Pool has {pretty_ether_and_wei(ALG_datatoken.balanceOf(wallet.address), ALG_datatoken.symbol())}.")

Pool has 1E+2 ARTGEN0 (100000000000000000000 wei).


## Alice publishes algorithm

Ocean Protocol provides some basic Dockerfiles in their [profile](https://hub.docker.com/r/oceanprotocol/algo_dockers) on Dockerhub. In the third notebook, we used the PyTorch library to train a simple generative model, which isn't installed in the Ocean Protocol containers. Hence, we will use a custom Dockerfile that we have pushed to our new Algovera profile on [Dockerhub](https://hub.docker.com/r/algovera/algo_dockers/tags). We use the image name and tag in the `container` part of the algorithm metadata.
This docker image needs to have basic support for dependency installation e.g. in the case of Python, OS-level library installations, pip installations etc.
Take a look at the [Ocean tutorials](https://docs.oceanprotocol.com/tutorials/compute-to-data-algorithms/) to learn more about docker image publishing.
For the url, we want raw text rather than html.

In [ ]:
# Important Parameters
image: str = "algovera/algo_dockers" 
tag: str = "generative-art"
url: str = "https://raw.githubusercontent.com/AlgoveraAI/generative-art/main/4-dcgan-c2d.py"
name: str = "Generative Algorithm: DCGAN"
author: str = "AlgoveraAI"

In [10]:
# Specify metadata and service attributes, for "GPR" algorithm script.
# In same location as Branin test dataset. GPR = Gaussian Process Regression.
ALG_metadata =  {
    "main": {
        "type": "algorithm",
        "algorithm": {
            "language": "python",
            "format": "docker-image",
            "version": "0.1",
            "container": {
              "entrypoint": "python $ALGO",
              "image": image,
              "tag": tag
            }
        },
        "files": [
    {
        "url": url,
        "index": 0,
        "contentType": "text/text",
      }
    ],
    "name": name, "author": author, "license": "CC0",
    "dateCreated": "2021-12-02T15:00:00Z"
    }
}

ALG_service_attributes = {
        "main": {
            "name": "ALG_dataAssetAccessServiceAgreement",
            "creator": wallet.address,
            "timeout": 3600 * 24,
            "datePublished": "2020-01-28T10:55:11Z",
            "cost": 1.0, # <don't change, this is obsolete>
        }
    }

In [17]:
# Set up a service provider. We'll use this same provider for ALG
from ocean_lib.data_provider.data_service_provider import DataServiceProvider
provider_url = DataServiceProvider.get_url(ocean.config)
# returns "http://localhost:8030"

# Calc DATA service compute descriptor
from ocean_lib.services.service import Service
from ocean_lib.common.agreements.service_types import ServiceTypes

# Calc ALG service access descriptor. We use the same service provider as DATA
ALG_access_service = Service(
    service_endpoint=provider_url,
    service_type=ServiceTypes.CLOUD_COMPUTE,
    attributes=ALG_service_attributes
)

# Publish metadata and service info on-chain
ALG_ddo = ocean.assets.create(
  metadata=ALG_metadata, # {"main" : {"type" : "algorithm", ..}, ..}
  publisher_wallet=wallet,
  services=[ALG_access_service],
  data_token_address=ALG_datatoken.address)
print(f"ALG did = '{ALG_ddo.did}'")

signing message with nonce 0: 0xE2e123115d5758Dd4C6F434E1c142e72ed8B2820, account=0x2338e4e94AEe1817701F65f2c751f7c844b0e43b
ALG did = 'did:op:E2e123115d5758Dd4C6F434E1c142e72ed8B2820'


In [10]:
# Publish metadata and service info on-chain
ALG_ddo = ocean.assets.create(
  metadata=ALG_metadata, # {"main" : {"type" : "algorithm", ..}, ..}
  publisher_wallet=wallet,
  service_descriptors=[ALG_access_service_descriptor],
  data_token_address=ALG_datatoken.address)
print(f"ALG did = '{ALG_ddo.did}'")

signing message with nonce 16: 0x1fDe09d7056F5A077e67C9170998855dbE0DE62D, account=0xD438208197a0C552ED04e5e51695EC695E30C284
ALG did = 'did:op:1fDe09d7056F5A077e67C9170998855dbE0DE62D'


At this point you will need to request that the data provider approves your code as a trusted algorithm on the dataset. For example, if you reach out to Algovera about one of our datasets, we will approve your request quickly.